Importing libraries
-

In [2]:
import pandas as pd
import os
import requests
import zipfile
import functions as f
import numpy as np

Defining folder paths for downloaded and cleaned data, as well as links for the surveys
-

In [4]:
raw_data_folder = r"C:\Users\mpola\OneDrive\Desktop\Career\Proje\Stack Overflow Surveys Analysis\raw data"
clean_data_folder = r"C:\Users\mpola\OneDrive\Desktop\Career\Proje\Stack Overflow Surveys Analysis\clean data"

if not os.path.exists(raw_data_folder):
    os.makedirs(raw_data_folder)

if not os.path.exists(clean_data_folder):
    os.makedirs(clean_data_folder)

survey_2022 = "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip"
survey_2023 = "https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip"
survey_2024 = "https://cdn.sanity.io/files/jo7n4k8s/production/262f04c41d99fea692e0125c342e446782233fe4.zip/stack-overflow-developer-survey-2024.zip"

links_list = [survey_2022, survey_2023, survey_2024]

Downloading the survey data and opening them with Pandas
-

In [6]:
# By using rsplit, we can get the filename as it is stored in the StackOverflow servers to assign to our downloaded zips

for i in links_list:
    f.download_data(i, raw_data_folder, i.rsplit('/',1)[1])

File downloaded from https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip under C:\Users\mpola\OneDrive\Desktop\Career\Proje\Stack Overflow Surveys Analysis\raw data\stack-overflow-developer-survey-2022.zip
File downloaded from https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip under C:\Users\mpola\OneDrive\Desktop\Career\Proje\Stack Overflow Surveys Analysis\raw data\stack-overflow-developer-survey-2023.zip
File downloaded from https://cdn.sanity.io/files/jo7n4k8s/production/262f04c41d99fea692e0125c342e446782233fe4.zip/stack-overflow-developer-survey-2024.zip under C:\Users\mpola\OneDrive\Desktop\Career\Proje\Stack Overflow Surveys Analysis\raw data\stack-overflow-developer-survey-2024.zip


In [7]:
# Getting all downloaded zipfile names from our download folder, in case of manual renaming inbetween steps for readability

surveys = [survey for survey in os.listdir(raw_data_folder)]

In [8]:
# Loading all dataframes into a list in the order we defined the links previously, and fetching the year of the survey result
# as a new column on the dataframe for when we merge all dataframes and split by question categories rather than year
df_list= []

for i,j in enumerate(surveys):
    df_list.append(f.read_zip(raw_data_folder + "\\" + str(j), "survey_results_public.csv"))

    # To get the year as a string, we iterate through the entries of our surveys list to get the digits of the strings in order.
    # This assumes the filenames only have the year as numbers, but in this specific case there really isn't any reason for the files to
    # have another number in their names so it works
    year = ''
    for k in str(j):
        if k.isdigit():
            year += k
    df_list[i]['SurveyYear'] = int(year)

Filtering unnecessary data and merging dataframes
-

In [10]:
# Getting the intersection of all dataframe columns in our dataframe list in order to discard any column that we cannot compare year-to-year

for i in range(len(df_list)):
    if i == 0:
        col0 = df_list[i].columns
    else :
        col0 = set(col0).intersection(df_list[i].columns)
    common_cols = list(col0)

In [11]:
# Dropping non-shared columns

for i in range(len(df_list)):
    df_list[i] = df_list[i][common_cols]

In [12]:
# Merging all dataframes in our list under a single master dataframe

for i,j in enumerate(df_list):
    if i == 0:
        df_master = j
    else :
        df_master = pd.concat([df_master, j])

In [13]:
# Running the detect_separator function to see which columns of this dataframe contain data that are checklists condensed into singular strings
# These columns are very annoying to work with as they are, so they will be split into multiple boolean columns at a later step

separator_list = f.detect_separator(df_master, 50, ';')

In [14]:
# By defining a new list of shared column names, we can erase columns from this list as we go along with our column categorization 
# so we have a clean list to fall back to, as well as a one-step solution to make a "misc" dataframe for any columns we deemed either
# unnecessary for our analysis or couldn't fit into a category

remaining_cols = common_cols
id_cols = ['ResponseId']
remaining_separator_cols = [x for x in separator_list]

In [15]:
remaining_cols

['WebframeHaveWorkedWith',
 'SurveyLength',
 'NEWSOSites',
 'TBranch',
 'Knowledge_3',
 'Knowledge_5',
 'Knowledge_6',
 'OfficeStackAsyncWantToWorkWith',
 'TimeAnswering',
 'Currency',
 'EdLevel',
 'MiscTechHaveWorkedWith',
 'ResponseId',
 'WebframeWantToWorkWith',
 'Knowledge_1',
 'SOVisitFreq',
 'LearnCode',
 'SurveyYear',
 'Employment',
 'ToolsTechHaveWorkedWith',
 'OfficeStackSyncWantToWorkWith',
 'Frequency_1',
 'ProfessionalTech',
 'LanguageWantToWorkWith',
 'Knowledge_4',
 'CompTotal',
 'SurveyEase',
 'Knowledge_2',
 'PlatformHaveWorkedWith',
 'OpSysProfessional use',
 'BuyNewTool',
 'OpSysPersonal use',
 'DatabaseHaveWorkedWith',
 'ICorPM',
 'NEWCollabToolsHaveWorkedWith',
 'PurchaseInfluence',
 'NEWCollabToolsWantToWorkWith',
 'TimeSearching',
 'SOPartFreq',
 'SOAccount',
 'RemoteWork',
 'MainBranch',
 'DevType',
 'Age',
 'WorkExp',
 'YearsCode',
 'OfficeStackAsyncHaveWorkedWith',
 'SOComm',
 'Frequency_3',
 'YearsCodePro',
 'CodingActivities',
 'MiscTechWantToWorkWith',
 'Lea

In [16]:
remaining_separator_cols

['WebframeHaveWorkedWith',
 'NEWSOSites',
 'OfficeStackAsyncWantToWorkWith',
 'MiscTechHaveWorkedWith',
 'WebframeWantToWorkWith',
 'LearnCode',
 'Employment',
 'ToolsTechHaveWorkedWith',
 'OfficeStackSyncWantToWorkWith',
 'ProfessionalTech',
 'LanguageWantToWorkWith',
 'PlatformHaveWorkedWith',
 'OpSysProfessional use',
 'BuyNewTool',
 'OpSysPersonal use',
 'DatabaseHaveWorkedWith',
 'NEWCollabToolsHaveWorkedWith',
 'NEWCollabToolsWantToWorkWith',
 'DevType',
 'OfficeStackAsyncHaveWorkedWith',
 'CodingActivities',
 'MiscTechWantToWorkWith',
 'LearnCodeOnline',
 'DatabaseWantToWorkWith',
 'LanguageHaveWorkedWith',
 'ToolsTechWantToWorkWith',
 'OfficeStackSyncHaveWorkedWith',
 'PlatformWantToWorkWith']

In [17]:
# To start off, we organize the columns about the tech respondents have worked or want to work with into their own folders, as 
# we have a lot of dataframes and letting them all sit in one folder makes it harder to work with them outside of Python

HaveWorkedWith = [x for x in remaining_cols if 'HaveWorkedWith' in x]
WantToWorkWith = [x for x in remaining_cols if 'WantToWorkWith' in x]


if not os.path.exists(clean_data_folder + '\\HaveWorkedWith'):
    os.makedirs(clean_data_folder + '\\HaveWorkedWith')

if not os.path.exists(clean_data_folder + '\\WantToWorkWith'):
    os.makedirs(clean_data_folder + '\\WantToWorkWith')


for i,j in enumerate(HaveWorkedWith):
    df_HaveWorkedWith_temp, remaining_cols = f.columns_bucket(df_master, remaining_cols, [j], id_cols)
    if j in remaining_separator_cols:
        remaining_separator_cols.remove(j)
    df_HaveWorkedWith_temp.to_csv(f'{clean_data_folder}\\HaveWorkedWith\\df_' + j + '.csv', index=False)
    

for i,j in enumerate(WantToWorkWith):
    df_WantToWorkWith_temp, remaining_cols = f.columns_bucket(df_master, remaining_cols, [j], id_cols)
    if j in remaining_separator_cols:
        remaining_separator_cols.remove(j)
    df_WantToWorkWith_temp.to_csv(f'{clean_data_folder}\\WantToWorkWith\\df_' + j + '.csv', index=False)

In [18]:
# Then we define a third folder to contain all the other dataframes that correspond to columns that contain semicolon-separated
# strings as boolean checklists.


bool_dfs_folder = '\\BooleanDataframes'

if not os.path.exists(clean_data_folder + bool_dfs_folder):
    os.makedirs(clean_data_folder + bool_dfs_folder)

temp_list = []
for i,j in enumerate(remaining_separator_cols):
    df_temp, remaining_cols = f.columns_bucket(df_master, remaining_cols, [j], id_cols)
    temp_list.append(j)
    df_temp.to_csv(f'{clean_data_folder}\\{bool_dfs_folder}\\df_' + j + '.csv', index=False)

remaining_separator_cols = [x for x in remaining_separator_cols if x not in temp_list]

In [19]:
# Finally, we have all the columns that are not in checklist format. These columns correlate with each other in certain ways, and
# rather than storing each column in its own dataframe it makes sense here to do some manual work to decipher the surveys and categorize them based
# on these correlations.

# To get a better idea of which question in the survey each column corresponds to, it makes sense to get a list of unique entries of each
# column, which we then choose an arbitrary entry from and search the survey manually, as the data given does not include the
# information for corresponding questions outside of the survey pdf itself.
# However, these codes are not necessary for the functionality of this project otherwise

df_uniques = []
for column in remaining_cols:
    df_uniques.append(list(df_master[column].unique()))

col_labels = [[a] + b for a,b in zip(remaining_cols, df_uniques)]

In [20]:
BasicInfo = ['MainBranch', 'RemoteWork', 'Age', 'SurveyYear']


df_BasicInfo, remaining_cols = f.columns_bucket(df_master, remaining_cols, BasicInfo, id_cols)
df_BasicInfo.to_csv(f'{clean_data_folder}\\df_BasicInfo.csv', index=False)

In [21]:
EduWork = ['EdLevel', 'YearsCode', 'YearsCodePro', 'WorkExp', 'OrgSize', 'Country', 'Currency', 'ConvertedCompYearly']


df_EduWork, remaining_cols = f.columns_bucket(df_master, remaining_cols, EduWork, id_cols)
df_EduWork.to_csv(f'{clean_data_folder}\\df_EduWork.csv', index=False)

In [22]:
SOInfo = ['SOVisitFreq', 'SOPartFreq', 'SOComm', 'SOAccount']


df_SOInfo, remaining_cols = f.columns_bucket(df_master, remaining_cols, SOInfo, id_cols)
df_SOInfo.to_csv(f'{clean_data_folder}\\df_SOInfo.csv', index=False)

In [23]:
ProfDevSeries = [x for x in remaining_cols if 'Knowledge_' in x]

for x in remaining_cols:
     if 'Frequency_' in x:
        ProfDevSeries.append(x)

ProfDevSeries.append('TimeSearching')
ProfDevSeries.append('TimeAnswering')


df_ProfDevSeries, remaining_cols = f.columns_bucket(df_master, remaining_cols, ProfDevSeries, id_cols)
df_ProfDevSeries.to_csv(f'{clean_data_folder}\\df_ProfDevSeries.csv', index=False)

In [24]:
df_Misc = df_master[remaining_cols]
df_Misc.to_csv(f'{clean_data_folder}\\df_Misc.csv', index=False)

Data Cleaning and Optimization
-

In [55]:
# At this point we have a lot of csv files that have plenty of columns with NaN values, as well as csv files containing dataframes that
# all have only boolean columns outside of the ResponseId columns. The first problem is easy to solve using dropna with no issues,
# but the second problem has multiple situational fixes that may or may not be preferred based on the dataframe we apply the fix to

# The first step is to dropna the normal dataframes with mixed column types.

files_list = os.listdir(clean_data_folder)
files_list_df = [item for item in files_list if item.startswith('df_')]

for csv in files_list_df:
    df_temp = pd.read_csv(f'{clean_data_folder}\\{csv}')
    df_temp['ResponseId'] = np.arange(df_temp.shape[0]).astype(int)
    df_temp.dropna(axis=0, thresh=2, inplace=True)
    
    for column in separator_list:
        # This for loop would be useful, if not for our categorization of dataframes into folders. It is still left here
        # on the off-chance we made an oversight or go back to a "all files in one folder" format later on
        if column in df_temp:
            df_temp = f.split_string_checklist(df_temp, column, ';')
    df_temp.to_csv(f'{clean_data_folder}\\{csv}', index=False)

In [27]:
# When it comes to the dataframes that store checklist answer (ie. select all that apply)
# data as strings with semicolon separators, we split these into
# a multitude of boolean columns. This splitting makes the data easier to work with, but can increase filesizes depending on
# how many different possible entires there were on the checklist

files_list = os.listdir(clean_data_folder)
folders_list = [item for item in files_list if not item.startswith('df_')]

for folder in folders_list:
    files_list2 = os.listdir(clean_data_folder + '\\' + folder)
    files_list2_df = [item for item in files_list2 if item.startswith('df_')]
    
    for csv_name in files_list2_df:
        df_temp = pd.read_csv(f'{clean_data_folder}\\{folder}\\{csv_name}')
        
        df_temp['ResponseId'] = np.arange(df_temp.shape[0]).astype(int)
        df_temp.dropna(axis=0, thresh=2, inplace=True)
        
        for column in separator_list:
            if column in df_temp:
                df_temp = f.split_string_checklist(df_temp, column, ';')

        df_temp.to_csv(f'{clean_data_folder}\\{folder}\\{csv_name}', index=False)

        col_name = csv_name.lstrip('df_').rstrip('.csv')
        df_temp = f.table_stack(df_temp, id_cols[0], col_name)

        # In some cases, these boolean checklist dataframes can have a majority of rows where only one column is set to True, with
        # dozens set to false. In such cases, we end up with nearly 230k rows and dozens of columns that convey data that could just
        # as easily be represented with more rows, but only two columns: one to list ResponseId (which will no longer be unique
        # per row) and one to list the occurences of answers as strings.

        # If working strictly inside Python, we could use a dictionary to store the different possible answers, and have our
        # second column list only a numeric identifier that correspond to the answer, but such a simplification
        # makes the data harder to configure in external programs to work with.

        # As this simplification does not necessarily make the dataframes faster or easier to work with, we store these in a 
        # separate folder with separate .csv files so that we can pick and choose which variant we use depending
        # on whichever one is more convenient for the program we use them with
        if not os.path.exists(f'{clean_data_folder}\\{folder}\\stacked'):
            os.makedirs(f'{clean_data_folder}\\{folder}\\stacked')
        
        df_temp.to_csv(f'{clean_data_folder}\\{folder}\\stacked\\{csv_name}', index=False)


In [57]:
df_temp = pd.read_csv(f'{clean_data_folder}\\df_EduWork.csv')
df_temp

,EdLevel,YearsCode,YearsCodePro,WorkExp,OrgSize,Country,Currency,ConvertedCompYearly,ResponseId
0,NaN,NaN,NaN,NaN,NaN,Canada,CAD\tCanadian dollar,NaN,1
1,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",14,5,NaN,20 to 99 employees,United Kingdom of Great Britain and Northern I...,GBP\tPound sterling,40205.0,2
2,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",20,17,NaN,100 to 499 employees,Israel,ILS\tIsraeli new shekel,215232.0,3
3,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",8,3,NaN,20 to 99 employees,United States of America,USD\tUnited States dollar,NaN,4
4,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",15,NaN,NaN,NaN,Germany,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...
220523,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",4,7,NaN,"1,000 to 4,999 employees",India,INR\tIndian rupee,NaN,227880
220524,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",38,24,NaN,20 to 99 employees,Belgium,EUR European Euro,NaN,227883
220525,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,3,NaN,2 to 9 employees,NaN,NaN,NaN,227884
220526,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",9,5,NaN,"1,000 to 4,999 employees",NaN,NaN,NaN,227886
